## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Данные представляют собой таблицу, которую нужно будет исследовать на пропуски, дубликаты, неверные типы и другие ошибочные или аномальные записи, затем будет произведена категоризация данных для выделения групп заемщиков по различным признакам. Для ответа на поставленные вопросы из общего набора данных будут выделены поднаборы для более наглядного анализа.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Подгрузим необходимые библиотеки и откроем файл с данными

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/datasets/data.csv')    # откроем файл
print(data.info())    # выведем общую информацию по загруженным данным

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Посмотрим первые 10 строк таблицы

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Выводы

- Таблица имеет 21525 строк и 12 столбцов.
- В столбцах days_employed и total_income пропуски, которые нужно будет обработать.
- Колонки содержат данные разных типов - как числовые, так и строки.
- В колонке с общим трудовым стажем для некоторых значений приведен стаж со знаком "-".

### Шаг 2. Предобработка данных

Исправим отрицательные значения в столбцах со стажем и количеством детей на положительные:

In [3]:
data['days_employed'] = np.abs(data['days_employed'])
data['children'] = np.abs(data['children'])

### 2.1 Обработка пропусков

__Посчитаем количество строк с пропущенными значениями в каждом столбце:__

In [4]:
for i in data.columns:
    if data[data[i].isnull()].shape[0]:
        print('Количество пропущенных значений в столбце {} равно {}'.format(i, data[data[i].isnull()].shape[0]))

Количество пропущенных значений в столбце days_employed равно 2174
Количество пропущенных значений в столбце total_income равно 2174


В столбцах days_employed и total_income по 2174 пропуска, так как тип данных в ячейках этих столбцов float64, значит надо привести пропущенный тип данных в этих ячейках к числовому виду.

Начнем с общего дохода. Надо выяснить, какой разброс в ежемесячных доходах разных категорий граждан.

In [5]:
# например, какие зарплаты у сотрудников 

print('Максимальная зарплата у сотрудника равна {}'.format(np.max(data['total_income'][data['income_type'] == 'сотрудник'])))
print('Минимальная зарплата у сотрудника равна {}'.format(np.min(data['total_income'][data['income_type'] == 'сотрудник'])))
print('Средняя зарплата у сотрудника равна {}'.format(np.mean(data['total_income'][data['income_type'] == 'сотрудник'])))

Максимальная зарплата у сотрудника равна 1726276.0143316737
Минимальная зарплата у сотрудника равна 21367.648356486974
Средняя зарплата у сотрудника равна 161380.26048788553


Разброс в 80 раз... Возьмем медиану, но только не учитывая nan значения

In [6]:
print('Медианная зарплата у сотрудника равна {}'.format(np.median(data['total_income'][(data['income_type'] == 'сотрудник') & 
                                                                                       (~data['total_income'].isnull())])))

Медианная зарплата у сотрудника равна 142594.39684740017


Учтем еще и образование для более точного значения медианы. Для начала приведем значения в education к нижнему регистру:

In [7]:
# приведем значения к нижнему регистру
data['education'] = data['education'].str.lower()

In [8]:
# напишем функцию, которая будет принимать имя столбца с пропущенными значениями и менять пропуски на медианные значения по типу
# занятости и образованию
def treatment_nan(col_name):
    # из создадим сводную таблицу с медианными значениями для каждого типа занятости и уровня образования из которой будем брать
    # значения медиан занятость-образование и подставлять на места пропусков
    data_not_null = data[~data[col_name].isnull()].pivot_table(col_name, index='income_type', columns='education')
    
    #Заполним полученными медианами для каждого типа занятости и образования пропуски в таблице:
    for income_type in data_not_null.index:
        for education in data_not_null.columns:
            if not np.isnan(data_not_null.loc[income_type, education]):
                data.loc[data[col_name].isnull() & (data['income_type'] == income_type)
                     & (data['education'] == education), col_name] = data_not_null.loc[income_type, education]

Заполним полученными медианами для каждого типа занятости и образования пропуски в таблице:

In [9]:
for i in ('days_employed', 'total_income'):
    treatment_nan(i)

Выведем информацию по всей таблице

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


__Все NaN значения заменены__

### Вывод

Пропущенные значения в общем стаже и ежемесячном доходе заполнены медианами из-за большого разброса между экстремумами в разных категориях 'income_type'.


### 2.2 Замена типа данных

In [11]:
for i in ('days_employed', 'total_income'):
    data[i] = data[i].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [12]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

1. Изменение типа прозведено с помощью astype.
2. Можно было использовать pd.to_numeric(data['col'])
3. Также можно было использовать np.astype('int')

### 2.3 Обработка дубликатов

Посчитаем количество дубликатов:

In [13]:
print('Количество дубликатов в таблице: {}'.format(data[data.duplicated()].shape[0]))

Количество дубликатов в таблице: 71


Нужно проверить регистры текста в столцах 'education', 'family_status', 'income_type' и 'purpose' и привести к одному регистру

In [14]:
for i in ('family_status', 'income_type', 'purpose'):
    data[i] = data[i].str.lower()

In [15]:
print('Количество дубликатов в таблице: {}'.format(data[data.duplicated()].shape[0]))

Количество дубликатов в таблице: 71


Вот теперь можно удалять, так как после приведения к регистру мы выявили все дубликаты:

In [16]:
print('Размер таблицы до удаления дубликатов {}'.format(data.shape))

data = data.drop_duplicates().reset_index(drop=True)    # удаляем дубликаты и обновляем индексы

print('Размер таблицы до удаления дубликатов {}'.format(data.shape))

Размер таблицы до удаления дубликатов (21525, 12)
Размер таблицы до удаления дубликатов (21454, 12)


### Вывод

1. После приведения к нижнему регистру строк столбца 'education' был выявлен 71 дубликат строк, которые были удалены для корректности дальнейшего анализа.
2. Поиск дубликатов осуществлен с помощью метода duplicated().
3. Возможно, дубликаты появились в следствии ошибкок ведения БД, таких как не согласованный регистр строковых данных или случайное дублирование строк.

### 2.4 Лемматизация

Посмотрим на данные в столбце с целями получения кредита

In [17]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Сохраним список целей кредита:

In [18]:
list_purpose = data['purpose'].to_list()

C помощью лемматизации посмотрим, какие слова встречаются чаще:

In [19]:
# импортируем класс Mystem из библиотеке pymystem3 и Counter для подсчета уникальных слов
from pymystem3 import Mystem
from collections import Counter

lemmas = Mystem().lemmatize(' '.join(list_purpose))
Counter(lemmas).most_common()

[(' ', 55023),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('подержать', 853),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('приобретение', 461),
 ('профильный', 436),
 ('подержанный', 111),
 ('\n', 1)]

### Вывод

1. Из подсчета уникальных слов в целях взятия кредита становится понятно, что основные цели это:
  1. Недвижимость
  2. Автомобиль
  3. Образование
  4. Свадьба
2. Создание категорий из набора целей кредита сделаем при ответе на последний вопрос в самом конце.

### 2.5 Категоризация данных

Разделим клиентов на возрастные группы:
- меньше 18 лет - дети;
- от 19 до 64 - взрослые;
- от 65 - пенсионеры
Это поможет выявить группу основных заемщиков банка

In [20]:
# напишем функцию для определения категории по возрасту
def age_group(age):    
    if age < 18:
        return 'дети'
    elif age > 64:
        return 'пенсионеры'
    else:
        return 'взрослые'

In [21]:
data['age_group'] = data['dob_years'].apply(age_group)

In [22]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,взрослые
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,взрослые
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослые


Добавим категории уровней дохода. Для этого посмотрим распределение доходов:

In [23]:
data['total_income'].describe()

count    2.145400e+04
mean     1.674864e+05
std      9.827834e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.523685e+05
75%      1.983070e+05
max      2.265604e+06
Name: total_income, dtype: float64

Введем следующие категории:
- менее 108000 - 'низкий доход'
- более 108000, но менее 195000 - 'средний доход'
- более 195000 - 'высокий доход'

In [24]:
def totinc_group(income):    
    if income < 108000:
        return 'низкий доход'
    elif income > 195000:
        return 'высокий доход'
    else:
        return 'средний доход'

In [25]:
# запишем категории дохода в отдельный столбец 'income_group'
data['income_group'] = data['total_income'].apply(totinc_group)

In [26]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,взрослые,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослые,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,взрослые,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,взрослые,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослые,средний доход


In [27]:
# создадим сводную таблицу, чтобы узнать средний доход для разных возрастных групп
data.pivot_table('total_income', index='income_group', columns='age_group').round(1)

age_group,взрослые,дети,пенсионеры
income_group,,,
высокий доход,283137.6,251237.8,279363.5
низкий доход,81578.4,81877.2,75516.8
средний доход,149411.0,150769.0,143062.4


Из сводной таблицы видно, что средние уровни доходов для разных возрастных групп примерно одинаковы.

Выделим следующие "словари" для ответов на итоговые вопросы:

In [28]:
# словарь с данными по количеству детей и возрату кредита в срок
data_first_qw = data[['children', 'debt']]

# словарь с данными по семейному положению и возрату кредита в срок
data_second_qw = data[['family_status', 'debt']]

# словарь с данными по уровню дохода и возрату кредита в срок
data_third_qw = data[['income_group', 'debt']]

# словарь с данными по цели кредита и возрату кредита в срок
data_forth_qw = data[['purpose', 'debt']]

### Вывод

1. В таблицу были добавлены категории по возрасту и доходу заемщиков, так как в этих данных слишком большой диапазон значений и проще поделить их на группы.
2. Были выделены словари с данными, которые можно отдельно анализировать в дальнейшем исследовании.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим на соответствующий словарь:

In [29]:
data_first_qw.head()

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0


Сгруппируем таблицу по наличию детей и посчитаем количество кредитов и просрочек по каждому количеству:

In [30]:
data_first_group = data_first_qw.pivot_table('debt', 'children', aggfunc={'debt': 'sum', 'children': 'count'})
# переименуем столбец c количеством просрочек для каждого количества детей в data_first_group
data_first_group.columns = ['count_debt', 'debt']
# добавим столбец с процентным соотношением просрочки / количество
data_first_group['perc'] = ((data_first_group['debt'] / data_first_group['count_debt']) * 100).round(1)
data_first_group

,count_debt,debt,perc
children,,,
0,14091,1063,7.5
1,4855,445,9.2
2,2052,194,9.5
3,330,27,8.2
4,41,4,9.8
5,9,0,0.0
20,76,8,10.5


In [31]:
# расчитаем, на сколько процентов возрастет риск, если у заемщика есть дети
(1 - data_first_group['perc'][0] / (data_first_group['perc'][1:].mean())) * 100

4.661016949152552

### Вывод

1. Зависимость есть - с увеличение количества детей возрастает риск невозврата кредита в срок.
2. Риск невозврата кредита для людей с детьми в среднем на 4,66 % больше, чем у людей без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
data_second_qw.head()

,family_status,debt
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,0


Определим функцию, чтобы не дублировать код, написаный выше. Функция пригодится для следующих вопросов:

In [33]:
def calc_perc(data, name_col):
    data_group = data.pivot_table('debt', name_col, aggfunc={'debt': 'sum', name_col: 'count'})
    # переименуем столбец c количеством просрочек для каждого количества детей в data_first_group
    data_group.columns = ['debt', 'count_debt', ]
    # добавим столбец с процентным соотношением просрочки / количество
    data_group['perc'] = ((data_group['debt'] / data_group['count_debt']) * 100).round(1)
    return data_group

In [34]:
data_second_group = calc_perc(data_second_qw, 'family_status')
data_second_group

,debt,count_debt,perc
family_status,,,
в разводе,85,1195,7.1
вдовец / вдова,63,959,6.6
гражданский брак,388,4151,9.3
женат / замужем,931,12339,7.5
не женат / не замужем,274,2810,9.8


### Вывод

1. Чаще всего задолжность по кредиту имеют не женатые/ не замужние (9.8 %) и люди, живущие в гражданском браке (9.3 %).
2. Реже имеют задолжность вдовы (6.6 %) и люди в разводе (7.1 %). Возможно, это связано с желанием скорее закрыть общие долги.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос будем использовать категорированные данные уровней дохода.

In [35]:
data_third_qw.head()

,income_group,debt
0,высокий доход,0
1,средний доход,0
2,средний доход,0
3,высокий доход,0
4,средний доход,0


In [36]:
# воспользуемся функцией calc_perc
data_third_group = calc_perc(data_third_qw, 'income_group')
data_third_group

,debt,count_debt,perc
income_group,,,
высокий доход,397,5690,7.0
низкий доход,431,5411,8.0
средний доход,913,10353,8.8


### Вывод

1. Как и ожидалось, люди с высоким уровнем дохода имеют меньше просрочек.
2. Люди со средним доходом чаще кредиты берут (как люди с низким и высоким доходом вместе взятые), но количество просрочек по этим кредитам выше.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
data_forth_qw.head()

,purpose,debt
0,покупка жилья,0
1,приобретение автомобиля,0
2,покупка жилья,0
3,дополнительное образование,0
4,сыграть свадьбу,0


In [38]:
# воспользуемся функцией calc_perc
data_forht_group = calc_perc(data_forth_qw, 'purpose')
data_forht_group.head(10)

,debt,count_debt,perc
purpose,,,
автомобили,44,478,9.2
автомобиль,42,494,8.5
высшее образование,40,452,8.8
дополнительное образование,38,460,8.3
жилье,46,646,7.1
заняться высшим образованием,43,496,8.7
заняться образованием,39,408,9.6
на покупку автомобиля,44,471,9.3
на покупку подержанного автомобиля,36,478,7.5


Теперь необходимо привести эти данные к 4 основным категориям:
  1. Недвижимость
  2. Автомобиль
  3. Образование
  4. Свадьба

In [39]:
new_index = []     # в этот список будут записаны новые индексы для data_forht_group

m = Mystem()       # создадим объект Mystem

# запишем лемматизированные значения в lemmat_list
lemmat_list = []
for i in data_forht_group.index:
    lemmat_list.append(m.lemmatize(i))

# заполним словарь new_index стандартизированными названиями категорий
for i in lemmat_list:
    if 'свадьба' in ''.join(i):
        new_index.append('на свадьбу')
    elif 'автомобиль' in ''.join(i):
        new_index.append('на автомобиль')
    elif 'образование' in ''.join(i):
        new_index.append('на образование')
    else:
        new_index.append('на недвижимость')

In [40]:
# теперь заменим цель в data_forht_group на категории
data_forht_group.index = new_index

In [41]:
# сделаем индекс столбцом
data_forht_group = data_forht_group.reset_index()

In [42]:
# изменим названия столбцов
data_forht_group.columns = ['purpose', 'debt', 'count_val', 'perc_debt']

In [43]:
data_forht_group.head()

,purpose,debt,count_val,perc_debt
0,на автомобиль,44,478,9.2
1,на автомобиль,42,494,8.5
2,на образование,40,452,8.8
3,на образование,38,460,8.3
4,на недвижимость,46,646,7.1


Теперь можно опять сгруппировать данные по категориям, расчитав средний процент одинаковых категорий целей кредита:

In [44]:
data_forht_group = data_forht_group.groupby('purpose')['debt', 'count_val'].sum()

# добавим столбец с процентами
data_forht_group['perc_debt'] = ((data_forht_group['debt'] / data_forht_group['count_val']) * 100).round(1)

Выведем итоговую таблицу

In [45]:
data_forht_group

,debt,count_val,perc_debt
purpose,,,
на автомобиль,403,4306,9.4
на недвижимость,782,10811,7.2
на образование,370,4013,9.2
на свадьбу,186,2324,8.0


### Вывод

1. Ответственнее всех в отношении просрочек по кредитам те, кто взял кредит на недвижимость (7,2 % просрочек). Возможно это связано с тем, что это чаще ипотеки и при просрочках можно остаться без жилья, что очень критично для любого человека.
2. Чаще имеют просрочки те, кто берут кредит на автомобиль (9,4 % просрочек) и на образование (9,2 %). На автомобиль обычно ниже ставки и кредиты дают охотнее, даже если у тебя маленький доход. Люди которые берут кредит на образования, возможно, не имеют хорошего дохода и поэтому могут иметь просрочки.

### Шаг 4. Общий вывод

В процессе анализа данных заказчика на предмет надежности заемщика были решены следующие задачи:
 - проанализирована общая информация по предоставленным данным;
 - выявлены ошибки в типах данных, типы были приведены к единому виду;
 - пропуски в столбцах с общим доходом и трудовым стажем были заменены медианными значениями по типу занятости и уровню образования;
 - строковые значения были приведены к единому регистру;
 - были обнаружены и ликвидированы дубликаты которые, возможно, появились в следствии ошибкок ведения БД, таких как не согласованный регистр строковых данных или случайное дублирование строк;
 - проведена процедура лемматизации для выявления основных целей получения кредита;
 - данные были категоризированы по возрастным группам и уровню дохода.

Для ответов на поставленные вопросы из набора данных были выделены словари с последующим созданием из них сводных таблиц и расчета процентных соотношений количества задолжностей к общему количеству взятых кредитов для каждой категории.

В результате работы были сделаны выводы, что чаще кредиты берут люди в браке, чаще цель кредита - покупка недвижимости. При этом, просрочки по кредитам обычно имеют, если:
1. Цель - покупка автомобиля или получение образования.
2. У заемщика есть дети.
3. Семейное положение - не женат/не замужем или в гражданском браке.

Полученые результаты можно использовать при построении модели кредитного скоринга. Также данный анализ можно провести на новых данных при соблюдении структуры и типов данных каждого столбца (также, возможно, было бы неплохо отобразить результаты из последних сводных таблиц в виде графиков для наглядной оценки полученныхе результатов).

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.